In [20]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Set up credentials
SERVICE_ACCOUNT_FILE = 'google-service-account.json'
SCOPES = ['https://www.googleapis.com/auth/forms', 'https://www.googleapis.com/auth/drive']

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Create Forms API service
forms_service = build('forms', 'v1', credentials=creds)

# Create a new form with only the title
form = {
    'info': {
        'title': 'Participant Information Form',
    }
}
result = forms_service.forms().create(body=form).execute()
form_id = result['formId']

# Update the form with description and questions
update_form = {
    'requests': [
        {
            'updateFormInfo': {
                'info': {
                    'description': 'Please fill out this form to participate in our study.'
                },
                'updateMask': 'description'
            }
        },
        {
            'createItem': {
                'item': {
                    'title': 'First Name',
                    'questionItem': {
                        'question': {
                            'required': True,
                            'textQuestion': {'paragraph': False}
                        }
                    }
                },
                'location': {'index': 0}
            }
        },
        {
            'createItem': {
                'item': {
                    'title': 'Last Name',
                    'questionItem': {
                        'question': {
                            'required': True,
                            'textQuestion': {'paragraph': False}
                        }
                    }
                },
                'location': {'index': 1}
            }
        },
        {
            'createItem': {
                'item': {
                    'title': 'Gmail',
                    'questionItem': {
                        'question': {
                            'required': True,
                            'textQuestion': {'paragraph': False}
                        }
                    },
                    'description': 'Please enter only the part before @gmail.com'
                },
                'location': {'index': 2}
            }
        },
        {
            'createItem': {
                'item': {
                    'title': 'Education',
                    'questionItem': {
                        'question': {
                            'required': True,
                            'choiceQuestion': {
                                'type': 'RADIO',
                                'options': [
                                    {'value': 'No Higher Education Degree'},
                                    {'value': 'Undergraduate Degree (Bachelor\'s Degree)'},
                                    {'value': 'Postgraduate Degree (Masters, MBA, Diplomas etc.)'},
                                    {'value': 'Doctoral Degree (Ph.D., M.D., J.D. etc.)'}
                                ]
                            }
                        }
                    }
                },
                'location': {'index': 3}
            }
        },
        {
            'createItem': {
                'item': {
                    'title': 'Status',
                    'questionItem': {
                        'question': {
                            'required': True,
                            'choiceQuestion': {
                                'type': 'RADIO',
                                'options': [
                                    {'value': 'Student'},
                                    {'value': 'Professional'}
                                ]
                            }
                        }
                    }
                },
                'location': {'index': 4}
            }
        }
    ]
}

forms_service.forms().batchUpdate(formId=form_id, body=update_form).execute()

# Create Drive API service to share the form
drive_service = build('drive', 'v3', credentials=creds)

# Share the form with your email
def share_form(file_id, email):
    drive_service.permissions().create(
        fileId=file_id,
        body={'type': 'user', 'role': 'writer', 'emailAddress': email},
        fields='id'
    ).execute()

share_form(form_id, 'venvoooo@gmail.com')

print(f"Form created with ID: {form_id}")
print(f"You can access the form at: https://docs.google.com/forms/d/{form_id}/edit")
print(f"Form shared with venvoooo@gmail.com")

# Generate an email template
email_template = f"""
Subject: Invitation to Participate in Our Study

Dear Participant,

We invite you to take part in our study. To get started, please fill out the participant information form using the link below:

https://docs.google.com/forms/d/{form_id}/viewform

Your participation is greatly appreciated. If you have any questions, please don't hesitate to contact us.

Thank you for your time and cooperation.

Best regards,
The Research Team
"""

print("\nEmail Template:")
print(email_template)

Form created with ID: 1Caj33KnzOmnGEmiNGPpaow4Id6rSVlJmOOGE0TPhycw
You can access the form at: https://docs.google.com/forms/d/1Caj33KnzOmnGEmiNGPpaow4Id6rSVlJmOOGE0TPhycw/edit
Form shared with venvoooo@gmail.com

Email Template:

Subject: Invitation to Participate in Our Study

Dear Participant,

We invite you to take part in our study. To get started, please fill out the participant information form using the link below:

https://docs.google.com/forms/d/1Caj33KnzOmnGEmiNGPpaow4Id6rSVlJmOOGE0TPhycw/viewform

Your participation is greatly appreciated. If you have any questions, please don't hesitate to contact us.

Thank you for your time and cooperation.

Best regards,
The Research Team



In [35]:
#!/usr/bin/env python3

import random
import sys
from time import sleep, asctime, time as time_now

import requests
try:
    from fake_useragent import FakeUserAgent
except ImportError:
    print('This program needs fake-useragent to run')
    sys.exit(1)

from gforms import Form
from gforms.errors import InfiniteLoop, ClosedForm, ValidationError
from gforms.elements import Short, Radio

class Filler:
    def __init__(self):
        self.first_names = ['John', 'Jane', 'Alice', 'Bob', 'Charlie']
        self.last_names = ['Doe', 'Smith', 'Johnson', 'Brown', 'Davis']
        self.education_options = [
            'No Higher Education Degree',
            'Undergraduate Degree (Bachelor\'s Degree)',
            'Postgraduate Degree (Masters, MBA, Diplomas etc.)',
            'Doctoral Degree (Ph.D., M.D., J.D. etc.)'
        ]
        self.status_options = ['Student', 'Professional']

    def callback(self, elem, i, j):
        if isinstance(elem, Short):
            if elem.name == 'First Name':
                return random.choice(self.first_names)
            elif elem.name == 'Last Name':
                return random.choice(self.last_names)
            elif elem.name == 'Gmail':
                return f"{random.choice(self.first_names).lower()}{random.randint(100,999)}"
        elif isinstance(elem, Radio):
            if elem.name == 'Education':
                return random.choice(self.education_options)
            elif elem.name == 'Status':
                return random.choice(self.status_options)
        return None  # Skip other elements

def main():
    ua = FakeUserAgent()
    sess = requests.session()
    sess.headers['User-Agent'] = ua.chrome

    form_id = '1lGM68GWafSah1uViPS2wVAvHKRHYfFPngl0vT-SUx2Q'
    form_url = f'https://docs.google.com/forms/d/{form_id}/viewform'

    form = Form()
    form.load(form_url, session=sess)
    filler = Filler()

    print(form.to_str(2))

    for i in range(5):  # Submit 5 responses
        print(f'[{asctime()}] Submit {i+1}... ', end='')
        try:
            form.fill(filler.callback, fill_optional=True)
            form.submit(session=sess)
            print('OK')
        except ClosedForm:
            print('Form is closed')
            break
        except (InfiniteLoop, ValidationError, RuntimeError) as e:
            print(f'Failed: {e}')
        
        sleep(random.uniform(5, 10))  # Sleep between 5 to 10 seconds between submissions

    print("All submissions completed.")

if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        print('An error occurred:', e)

Participant Information Form
Please fill out this form to participate in our study.
  Page 1:
  ——————————————————————————————————————————————————————————
    Short*: First Name
  ——————————————————————————————————————————————————————————
    Short*: Last Name
  ——————————————————————————————————————————————————————————
    Short*: Gmail
    Please enter only the part before @gmail.com
  ——————————————————————————————————————————————————————————
    Radio*: Education
    ◯ No Higher Education Degree
    ◯ Undergraduate Degree (Bachelor's Degree)
    ◯ Postgraduate Degree (Masters, MBA, Diplomas etc.)
    ◯ Doctoral Degree (Ph.D., M.D., J.D. etc.)
  ——————————————————————————————————————————————————————————
    Radio*: Status
    ◯ Student
    ◯ Professional
[Wed Oct 16 16:51:47 2024] Submit 1... OK
[Wed Oct 16 16:51:53 2024] Submit 2... OK
[Wed Oct 16 16:52:02 2024] Submit 3... OK
[Wed Oct 16 16:52:13 2024] Submit 4... OK
[Wed Oct 16 16:52:21 2024] Submit 5... OK
All submissions comple

In [44]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from pprint import pprint

# Set up credentials
SERVICE_ACCOUNT_FILE = 'google-service-account.json'
SCOPES = [
    'https://www.googleapis.com/auth/forms.responses.readonly',
    'https://www.googleapis.com/auth/forms',
    'https://www.googleapis.com/auth/drive.readonly'
]

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Create Forms API service
forms_service = build('forms', 'v1', credentials=creds)

# Your form ID
form_id = '1lGM68GWafSah1uViPS2wVAvHKRHYfFPngl0vT-SUx2Q'

def get_form_responses():
    try:
        # Get form structure
        form = forms_service.forms().get(formId=form_id).execute()
        questions = {item['questionItem']['question']['questionId']: item['title'] 
                     for item in form['items'] if 'questionItem' in item}

        # Get form responses
        responses = forms_service.forms().responses().list(formId=form_id).execute()

        # Process and print responses
        print(f"Total responses: {len(responses.get('responses', []))}\n")
        for response in responses.get('responses', []):
            print(f"Response ID: {response['responseId']}")
            for question_id, answer in response['answers'].items():
                question_title = questions.get(question_id, "Unknown Question")
                answer_value = answer['textAnswers']['answers'][0]['value']
                print(f"  {question_title}: {answer_value}")
            print()
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == '__main__':
    get_form_responses()

An error occurred: <HttpError 404 when requesting https://forms.googleapis.com/v1/forms/1lGM68GWafSah1uViPS2wVAvHKRHYfFPngl0vT-SUx2Q?alt=json returned "Requested entity was not found". Details: "Requested entity was not found">


In [43]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Set up credentials
SERVICE_ACCOUNT_FILE = 'google-service-account.json'
SCOPES = [
    'https://www.googleapis.com/auth/forms',
    'https://www.googleapis.com/auth/drive'
]

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
creds = creds.with_quota_project(creds.project_id)

# Create Forms API service
forms_service = build('forms', 'v1', credentials=creds)
drive_service = build('drive', 'v3', credentials=creds)

# Your form ID
form_id = '1lGM68GWafSah1uViPS2wVAvHKRHYfFPngl0vT-SUx2Q'

def clear_form_responses():
    try:
        # Get the original form
        form = forms_service.forms().get(formId=form_id).execute()

        # Create a new form with just the title
        new_form = {
            'info': {
                'title': form['info']['title'],
            }
        }
        new_form = forms_service.forms().create(body=new_form).execute()
        new_form_id = new_form['formId']

        # Update the new form with description and questions
        update_form = {'requests': []}
        
        # Add description if it exists
        if 'description' in form['info']:
            update_form['requests'].append({
                'updateFormInfo': {
                    'info': {
                        'description': form['info']['description']
                    },
                    'updateMask': 'description'
                }
            })

        # Add questions
        for item in form['items']:
            update_form['requests'].append({
                'createItem': {
                    'item': item,
                    'location': {'index': len(update_form['requests']) - 1}
                }
            })

        forms_service.forms().batchUpdate(formId=new_form_id, body=update_form).execute()

        # Delete the old form
        drive_service.files().delete(fileId=form_id).execute()

        print(f"Old form deleted. New form created with ID: {new_form_id}")
        print(f"You can access the new form at: https://docs.google.com/forms/d/{new_form_id}/edit")

    except HttpError as error:
        print(f"An error occurred: {error}")

if __name__ == '__main__':
    clear_form_responses()

Old form deleted. New form created with ID: 1IrdDnkRBDOcT2Bbim5K57xE0uCgfuV9BZAPTCDp9tlU
You can access the new form at: https://docs.google.com/forms/d/1IrdDnkRBDOcT2Bbim5K57xE0uCgfuV9BZAPTCDp9tlU/edit
